In [1]:
import keras
from keras.models import Sequential, load_model
from keras.layers import LSTM, GRU, TimeDistributed, Bidirectional, LeakyReLU
from keras.layers import Dense, Dropout, Activation, Flatten,  Input, Reshape, GRU, CuDNNGRU
from keras.layers import Convolution1D, MaxPool1D, GlobalAveragePooling1D,concatenate,AveragePooling1D
from keras.utils import multi_gpu_model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from keras.models import Model
from keras.utils import multi_gpu_model
from keras import initializers, regularizers, constraints
from keras.layers import Layer

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
import librosa
import librosa.display
import os
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import keras
import datetime as dt
from datetime import datetime
from keras import layers
from keras import models
from keras import optimizers
from keras.applications.densenet import DenseNet121, DenseNet169
#from keras.applications.nasnet import NASNetLarge
# from keras_efficientnets import EfficientNetB7
from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints
from keras.layers.normalization import BatchNormalization 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras import backend as K

tf.set_random_seed(1234)
random.seed(100)

os.environ["CUDA_VISIBLE_DEVICES"]="1"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

def score_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def dot_product(x, kernel):
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    
class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)
        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
            self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)
        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)
        if self.bias:
            uit += self.b
        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)
        a = K.exp(ait)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

    
def cce_f1_loss(y_true, y_pred):
    return 1 + 0.1*keras.losses.categorical_crossentropy(y_true, y_pred) - keras.metrics.categorical_accuracy(y_true, y_pred)

# Find unique number of classes  
def get_unique_classes(input_directory,files):

    unique_classes=set()
    for f in files:
        g = f.replace('.mat','.hea')
        input_file = os.path.join(input_directory,g)
        with open(input_file,'r') as f:
            for lines in f:
                if lines.startswith('#Dx'):
                    tmp = lines.split(': ')[1].split(',')
                    for c in tmp:
                        unique_classes.add(c.strip())
    return sorted(unique_classes)

def one_hot_encoding(one_hot_vector,y, class2index):
    ind=class2index[y]
    one_hot_vector[ind]=1
    return one_hot_vector

# Search for multi-label subjects
def searching_overlap(input_directory,class2index, input_file_names):
    multiclasses=[]
    multisubjects=[]
    number = []
    for file in input_file_names:
        f=file
        g = f.replace('.mat','.hea')
        input_file = os.path.join(input_directory,g)
        with open(input_file,'r') as f:
            for lines in f:
                if lines.startswith('#Dx'):
                    tmp = lines.split(': ')[1].split(',')
                    if len(tmp)>1:
                        one_hot_vector = [0]*(len(class2index))
                        for c in tmp:
                            one_hot_vector = one_hot_encoding(one_hot_vector, c.strip(), class2index)
                        multiclasses.append(one_hot_vector)
                        multisubjects.append(g)
                        number.append(len(tmp))
    return multisubjects, multiclasses, number

def block_feature(sequence_en, minimum_len): 
    new_en = []
    if len(sequence_en) > minimum_len:  # 길이가 minimum보다 긴 경우
        start = random.randint(0,len(sequence_en)-minimum_len)
        #print(start)
        new_en = sequence_en[start:start+minimum_len]
    elif len(sequence_en) == minimum_len: # 길이가 minimum
        new_en = sequence_en
    else: 
        assert len(sequence_en) <= minimum_len
    return new_en

def exploratory_look(input_directory,file, class2index):
    classes = []
    f = file
    g = f.replace('.mat','.hea')
    input_file = os.path.join(input_directory,g)
    with open(input_file,'r') as f:
        for lines in f:
            if lines.startswith('#Dx'):
                tmp = lines.split(': ')[1].split(',')
                print(tmp, len(tmp))
    return tmp     

# Get classes of sorted file names
def get_labels(input_directory,file, class2index):
    f = file
    g = f.replace('.mat','.hea')
    input_file = os.path.join(input_directory,g)
    with open(input_file,'r') as f:
        for lines in f:
            if lines.startswith('#Dx'):
                tmp = lines.split(': ')[1].split(',')
                one_hot_vector = [0]*(len(class2index))
                for c in tmp:
                    one_hot_vector = one_hot_encoding(one_hot_vector, c.strip(), class2index)
                
    return one_hot_vector

def randextract_mels(curr_step, batch_size, data, mel_directory, class2index, minimum_len, x_mean_final, x_std_final):
    mel_files = []
    classes = []
    start = batch_size*curr_step
    end = batch_size*(curr_step+1)
    curr_file_indices = data[start:end]
    for file in curr_file_indices:
        tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
        clip_file = block_feature(tmp_file, minimum_len)
        #print(clip_file.shape)
        #clip_file = tmp_file[:minimum_len]
        clip_file -= x_mean_final
        clip_file /= x_std_final
        mel_files.append(clip_file)
        label = get_labels(input_directory, file, class2index)
        classes.append(label)
    concat = list(zip(mel_files, classes))
    random.shuffle(concat)
    mel_files, classes = zip(*concat)
    return mel_files, classes

def zeropadding_mels(curr_step, batch_size, data, mel_directory, class2index, minimum_len, x_mean_final, x_std_final):
    mel_files = []
    classes = []
    start = batch_size*curr_step
    end = batch_size*(curr_step+1)
    curr_file_indices = data[start:end]    
    for file in curr_file_indices:
        
        tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
        tmp_file -= x_mean_final
        tmp_file /= x_std_final        
#         print(tmp_file.shape)
        zero_padding = np.zeros((minimum_len-len(tmp_file), 12))
#         print(zero_padding.shape)
        clip_file = np.concatenate((zero_padding, tmp_file), axis=0)
#         print(clip_file)
#         clip_file = zero_padding + tmp_file
#         print(clip_file.shape)
#         clip_file = block_feature(tmp_file, minimum_len)
        #print(clip_file.shape)
        #clip_file = tmp_file[:minimum_len]

        mel_files.append(clip_file)
        label = get_labels(input_directory, file, class2index)
        classes.append(label)
    concat = list(zip(mel_files, classes))
    random.shuffle(concat)
    mel_files, classes = zip(*concat)
    return mel_files, classes    

def train(data_train, mel_directory, batch_size, class2index, minimum_len, x_mean_final, x_std_final): 
    loss=[]
    acc = []
    f1 = []

    total_steps = int(np.ceil(len(data_train)/batch_size))
    for curr_step in range(total_steps):
        batch_mels, batch_labels = zeropadding_mels(curr_step, batch_size, data_train, mel_directory, class2index, minimum_len, x_mean_final, x_std_final)
#         batch_mels, batch_labels = randextract_mels(curr_step, batch_size, data_train, mel_directory, class2index, minimum_len, x_mean_final, x_std_final)
        batch_mels = np.asarray(batch_mels)
        batch_labels = np.asarray(np.squeeze(batch_labels))
        train_tmp = model.train_on_batch(batch_mels, batch_labels)
        loss.append(train_tmp[0])
        acc.append(train_tmp[1])
        f1.append(train_tmp[2])

    loss = np.mean(np.array(loss))
    acc = np.mean(np.array(acc))
    f1 = np.mean(np.array(f1))
    return loss, acc, f1

def test(data, mel_directory, input_directory, class2index, minimum_len, model, x_mean_final, x_std_final):
    scores = []
    predicted_labels=[]
    accuracy=np.zeros(len(data))
    #total_loss=[]
    total_acc = 0
    total_f1 = 0
    y_true = []
    y_pred=[]    
    mel_files = []
    classes = []
    
    
    multi_files, _, _ = searching_overlap(input_directory,class2index, data)
    multi_acc = 0
    multi_y_true=[]
    multi_y_pred=[]       
    for file in data:
        tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
        steps = int(np.floor(tmp_file.shape[0]/minimum_len))
        mel_files = []
        for block in range(steps): # 128개씩 쪼갠 블럭 단위로 predict
            start = block*minimum_len
            end = (block+1)*minimum_len
            clip_file = tmp_file[start:end]
            clip_file -= x_mean_final
            clip_file /= x_std_final
            mel_files.append(clip_file)
        mel_files = np.asarray(mel_files)
        logit = model.predict(mel_files)
        logit = np.mean(logit, axis=0)
        
        
        
        pred = np.zeros(len(logit))
        for ii, label in enumerate(logit):
            if label >= 0.5:
                pred[ii] = 1
            else:
                pred[ii] = 0
        pred = pred.tolist()
        y_pred.append(pred)        
        label = get_labels(input_directory,file,class2index)
        y_true.append(label)
        
        if pred == label:
            acc = 1
        else:
            acc = 0
        total_acc += acc
        if file.replace('.mat', '.hea') in multi_files:
            multi_y_true.append(label)            
            multi_y_pred.append(pred)
            multi_acc += acc         
        #total_f1 += f1
    final_acc = total_acc / len(data)
    f1_classes = f1_score(y_true, y_pred, average=None)
    f1_micro = f1_score(y_true, y_pred, average='micro')
    
    multi_final_acc = multi_acc / len(multi_files)
    multi_f1_classes = f1_score(multi_y_true, multi_y_pred, average=None)
    multi_f1_micro = f1_score(multi_y_true, multi_y_pred, average='micro')    
    #final_f1 = total_f1 / i
#     return final_acc#, final_f1
    return final_acc, f1_classes, f1_micro, multi_final_acc, multi_f1_classes, multi_f1_micro
def test_zero(data, mel_directory, input_directory, class2index, minimum_len, model, x_mean_final, x_std_final):
    scores = []
    predicted_labels=[]
    accuracy=np.zeros(len(data))
    #total_loss=[]
    total_acc = 0
    total_f1 = 0
    y_true = []
    y_pred=[]    
    mel_files = []
    classes = []
    
    
    multi_files, _, _ = searching_overlap(input_directory,class2index, data)
    multi_acc = 0
    multi_y_true=[]
    multi_y_pred=[]       
    for file in data:
        tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
        tmp_file -= x_mean_final
        tmp_file /= x_std_final
        steps = int(np.floor(tmp_file.shape[0]/minimum_len))
        mel_files = []
#         for block in range(steps): # 128개씩 쪼갠 블럭 단위로 predict
            
        zero_padding = np.zeros((minimum_len-len(tmp_file), 12))
        clip_file = np.concatenate((zero_padding, tmp_file), axis=0)
#         print(clip_file.shape)
        

        mel_files.append(clip_file)
        mel_files = np.asarray(mel_files)
        logit = model.predict(mel_files)
        logit = np.mean(logit, axis=0)
        
        
        
        pred = np.zeros(len(logit))
        for ii, label in enumerate(logit):
            if label >= 0.5:
                pred[ii] = 1
            else:
                pred[ii] = 0
        pred = pred.tolist()
        y_pred.append(pred)        
        label = get_labels(input_directory,file,class2index)
        y_true.append(label)
        
        if pred == label:
            acc = 1
        else:
            acc = 0
        total_acc += acc
        if file.replace('.mat', '.hea') in multi_files:
            multi_y_true.append(label)            
            multi_y_pred.append(pred)
            multi_acc += acc         
        #total_f1 += f1
    final_acc = total_acc / len(data)
    f1_classes = f1_score(y_true, y_pred, average=None)
    f1_micro = f1_score(y_true, y_pred, average='micro')
    
    multi_final_acc = multi_acc / len(multi_files)
    multi_f1_classes = f1_score(multi_y_true, multi_y_pred, average=None)
    multi_f1_micro = f1_score(multi_y_true, multi_y_pred, average='micro')    
    #final_f1 = total_f1 / i
#     return final_acc#, final_f1
    return final_acc, f1_classes, f1_micro, multi_final_acc, multi_f1_classes, multi_f1_micro
batch_size = 32
minimum_len = 72000 #2880
epochs = 1000
# loss_function = 'binary_crossentropy'
loss_function = 'categorical_crossentropy'
# activation_function = 'softmax'
rootdir = '../'
date = datetime.today().strftime("%Y%m%d")
input_directory = os.path.join(rootdir, 'Training_WFDB')
mel_name = 'Raw_data_20200424' 
mel_directory = os.path.join(rootdir, mel_name)
results_directory = os.path.join(rootdir, 'results_'+date+'1stModel')
if not os.path.isdir(input_directory):
    os.mkdir(input_directory)
if not os.path.isdir(mel_directory):
    os.mkdir(mel_directory)
if not os.path.isdir(results_directory):
    os.mkdir(results_directory)
        
input_files = []
for f in os.listdir(input_directory):
    if os.path.isfile(os.path.join(input_directory, f)) and not f.lower().startswith('.') and f.lower().endswith('mat'):
        input_files.append(f)
input_file_names = sorted(input_files)

unique_classes = get_unique_classes(input_directory, input_files)
class2index = {}
for a, b in enumerate(unique_classes):
    class2index[b] = a
    
# classes_orig= [x.replace('.mat', '.hea') for x in input_file_names] # total subjects
# classes_multi, _, _ = searching_overlap(input_directory,class2index, input_file_names)
# classes_single = [x for x in classes_orig if x not in classes_multi]
# classes_single = [x.replace('.hea', '.mat') for x in classes_single]

# double-checking if classes_single have single-label
# a, b, c  = searching_overlap(input_directory,class2index,classes_single)

# we can safely use classes_single as input_file_names
# input_file_names = classes_single

np.shape(input_file_names)

x_mean_all = []
x_std_all = []
for file in input_file_names:
    x = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
    x_mean = [np.mean(x[:,0]), np.mean(x[:,1]), np.mean(x[:,2]), np.mean(x[:,3]), np.mean(x[:,4]), np.mean(x[:,5]),
             np.mean(x[:,6]), np.mean(x[:,7]), np.mean(x[:,8]), np.mean(x[:,9]), np.mean(x[:,10]), np.mean(x[:,11])]
    
    x_std = [np.std(x[:,0]), np.std(x[:,1]), np.std(x[:,2]), np.std(x[:,3]), np.std(x[:,4]), np.std(x[:,5]),
             np.std(x[:,6]), np.std(x[:,7]), np.std(x[:,8]), np.std(x[:,9]), np.std(x[:,10]), np.std(x[:,11])]
    #print(x_mean)
    x_mean_all.append(x_mean)
    x_std_all.append(x_std)
x_mean_final = np.mean(x_mean_all, axis=0)
x_std_final = np.mean(x_std_all, axis=0)
print(x_mean_final)

data, data_test = train_test_split(input_file_names, test_size = 0.2, train_size = 0.8, shuffle=True, random_state=1004)
data_train, data_val = train_test_split(data, test_size = 0.05, train_size = 0.95, shuffle=True, random_state=1004)
print(np.shape(data_train), np.shape(data_val), np.shape(data_test))
print(data_train[0], data_val[0], data_test[0])

main_input = Input(shape=(minimum_len,12), dtype='float32', name='main_input')
num_classes= 9 
branch_pred = []
x = Convolution1D(12, 3, padding='same')(main_input)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 24, strides = 2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 24, strides = 2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 24, strides = 2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 24, strides = 2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 48, strides = 2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
cnnout = Dropout(0.2)(x)
x = Bidirectional(CuDNNGRU(12, input_shape=(2250,12),return_sequences=True,return_state=False))(cnnout)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = AttentionWithContext()(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
main_output = Dense(num_classes,activation='sigmoid')(x)
model = Model(inputs=main_input, outputs=main_output)
model.summary()

model.compile(loss=loss_function,
              optimizer=optimizers.Adam(lr=1e-5),
              metrics=['acc', score_f1])

val_acc_sum=[]
train_loss_sum=[]
train_acc_sum=[]
val_loss_sum=[]
val_acc_min = 0
patience = 50
earlystop = 0
for num_epoch in range(epochs):
    random.shuffle(data_train)
    train_loss, train_acc, train_f1 = train(data_train, mel_directory, batch_size, class2index, minimum_len, x_mean_final, x_std_final)
    print('\nEpoch',num_epoch+1,'train_loss:',f'{train_loss:.3f}','train_acc:',f'{train_acc:.3f}','train_f1:',f'{train_f1:.3f}',"\t")
    model_output = "ecg_mel_E%02dL%.2f" % (num_epoch, train_loss)
    save_name = os.path.join(results_directory, model_output)
    final_acc, f1_classes, f1_micro,_,_,_ = test_zero(data_val, mel_directory, input_directory, class2index, minimum_len, model, x_mean_final, x_std_final)
#     final_acc, f1_classes, f1_micro,_,_,_ = test(data_val, mel_directory, input_directory, class2index, minimum_len, model, x_mean_final, x_std_final)
    if f1_micro > val_acc_min:
        val_acc_min = f1_micro
        earlystop = 0
        model.save(save_name)
    else: 
        earlystop += 1 
        if earlystop > patience:
            print("************Early stopped training due to non-improved performance**********")
#             break         
            
    print('\nValidation', num_epoch+1, 'valid_acc:',f'{f1_micro:.3f}', 'best_acc:',f'{val_acc_min:.3f}', "\t")


/usr/local/lib/python3.6/dist-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


[-0.0012702   0.00038759  0.00402317 -0.0029567   0.00051957  0.00116427
 -0.0008405  -0.00222516 -0.0019119  -0.00093502  0.00129789  0.00177598]
(5225,) (276,) (1376,)
A2355.mat A5949.mat A0349.mat




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 72000, 12)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 72000, 12)         444       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 72000, 12)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 72000, 12)         444       
_________________________________________________________________
leaky_re_lu_2 (LeakyR


Validation 1 valid_acc: 0.246 best_acc: 0.246 	

Epoch 2 train_loss: 2.351 train_acc: 0.174 train_f1: 0.267 	

Validation 2 valid_acc: 0.258 best_acc: 0.258 	

Epoch 3 train_loss: 2.348 train_acc: 0.175 train_f1: 0.263 	

Validation 3 valid_acc: 0.260 best_acc: 0.260 	

Epoch 4 train_loss: 2.337 train_acc: 0.153 train_f1: 0.251 	

Validation 4 valid_acc: 0.260 best_acc: 0.260 	

Epoch 5 train_loss: 2.314 train_acc: 0.148 train_f1: 0.245 	

Validation 5 valid_acc: 0.248 best_acc: 0.260 	

Epoch 6 train_loss: 2.300 train_acc: 0.150 train_f1: 0.246 	

Validation 6 valid_acc: 0.240 best_acc: 0.260 	

Epoch 7 train_loss: 2.292 train_acc: 0.163 train_f1: 0.251 	

Validation 7 valid_acc: 0.241 best_acc: 0.260 	

Epoch 8 train_loss: 2.285 train_acc: 0.160 train_f1: 0.252 	

Validation 8 valid_acc: 0.248 best_acc: 0.260 	

Epoch 9 train_loss: 2.276 train_acc: 0.167 train_f1: 0.257 	

Validation 9 valid_acc: 0.256 best_acc: 0.260 	

Epoch 10 train_loss: 2.272 train_acc: 0.171 train_f1: 0.257 	



Validation 74 valid_acc: 0.338 best_acc: 0.342 	

Epoch 75 train_loss: 1.964 train_acc: 0.334 train_f1: 0.341 	

Validation 75 valid_acc: 0.341 best_acc: 0.342 	

Epoch 76 train_loss: 1.965 train_acc: 0.350 train_f1: 0.341 	

Validation 76 valid_acc: 0.341 best_acc: 0.342 	

Epoch 77 train_loss: 1.961 train_acc: 0.343 train_f1: 0.341 	

Validation 77 valid_acc: 0.343 best_acc: 0.343 	

Epoch 78 train_loss: 1.963 train_acc: 0.338 train_f1: 0.340 	

Validation 78 valid_acc: 0.346 best_acc: 0.346 	

Epoch 79 train_loss: 1.955 train_acc: 0.337 train_f1: 0.342 	

Validation 79 valid_acc: 0.345 best_acc: 0.346 	

Epoch 80 train_loss: 1.953 train_acc: 0.350 train_f1: 0.343 	

Validation 80 valid_acc: 0.344 best_acc: 0.346 	

Epoch 81 train_loss: 1.946 train_acc: 0.352 train_f1: 0.344 	

Validation 81 valid_acc: 0.345 best_acc: 0.346 	

Epoch 82 train_loss: 1.947 train_acc: 0.347 train_f1: 0.344 	

Validation 82 valid_acc: 0.347 best_acc: 0.347 	

Epoch 83 train_loss: 1.939 train_acc: 0.355 t


Validation 146 valid_acc: 0.402 best_acc: 0.402 	

Epoch 147 train_loss: 1.785 train_acc: 0.437 train_f1: 0.392 	

Validation 147 valid_acc: 0.397 best_acc: 0.402 	

Epoch 148 train_loss: 1.782 train_acc: 0.445 train_f1: 0.391 	

Validation 148 valid_acc: 0.398 best_acc: 0.402 	

Epoch 149 train_loss: 1.786 train_acc: 0.440 train_f1: 0.393 	

Validation 149 valid_acc: 0.401 best_acc: 0.402 	

Epoch 150 train_loss: 1.771 train_acc: 0.448 train_f1: 0.394 	

Validation 150 valid_acc: 0.403 best_acc: 0.403 	

Epoch 151 train_loss: 1.777 train_acc: 0.435 train_f1: 0.394 	

Validation 151 valid_acc: 0.404 best_acc: 0.404 	

Epoch 152 train_loss: 1.776 train_acc: 0.440 train_f1: 0.394 	

Validation 152 valid_acc: 0.405 best_acc: 0.405 	

Epoch 153 train_loss: 1.772 train_acc: 0.445 train_f1: 0.395 	

Validation 153 valid_acc: 0.406 best_acc: 0.406 	

Epoch 154 train_loss: 1.766 train_acc: 0.444 train_f1: 0.395 	

Validation 154 valid_acc: 0.405 best_acc: 0.406 	

Epoch 155 train_loss: 1.773 

...


Validation 659 valid_acc: 0.664 best_acc: 0.664 	

Epoch 660 train_loss: 1.099 train_acc: 0.676 train_f1: 0.667 	

Validation 660 valid_acc: 0.653 best_acc: 0.664 	

Epoch 661 train_loss: 1.104 train_acc: 0.674 train_f1: 0.671 	

Validation 661 valid_acc: 0.663 best_acc: 0.664 	

Epoch 662 train_loss: 1.092 train_acc: 0.673 train_f1: 0.667 	

Validation 662 valid_acc: 0.657 best_acc: 0.664 	

Epoch 663 train_loss: 1.093 train_acc: 0.675 train_f1: 0.670 	

Validation 663 valid_acc: 0.654 best_acc: 0.664 	

Epoch 664 train_loss: 1.090 train_acc: 0.668 train_f1: 0.676 	

Validation 664 valid_acc: 0.658 best_acc: 0.664 	

Epoch 665 train_loss: 1.084 train_acc: 0.681 train_f1: 0.674 	

Validation 665 valid_acc: 0.669 best_acc: 0.669 	

Epoch 666 train_loss: 1.086 train_acc: 0.688 train_f1: 0.680 	

Validation 666 valid_acc: 0.663 best_acc: 0.669 	

Epoch 667 train_loss: 1.082 train_acc: 0.681 train_f1: 0.673 	

Validation 667 valid_acc: 0.664 best_acc: 0.669 	

Epoch 668 train_loss: 1.081 


Epoch 731 train_loss: 1.027 train_acc: 0.701 train_f1: 0.691 	

Validation 731 valid_acc: 0.695 best_acc: 0.698 	

Epoch 732 train_loss: 1.030 train_acc: 0.705 train_f1: 0.693 	

Validation 732 valid_acc: 0.688 best_acc: 0.698 	

Epoch 733 train_loss: 1.021 train_acc: 0.697 train_f1: 0.694 	

Validation 733 valid_acc: 0.689 best_acc: 0.698 	

Epoch 734 train_loss: 1.031 train_acc: 0.702 train_f1: 0.690 	

Validation 734 valid_acc: 0.690 best_acc: 0.698 	

Epoch 735 train_loss: 1.014 train_acc: 0.708 train_f1: 0.702 	

Validation 735 valid_acc: 0.694 best_acc: 0.698 	

Epoch 736 train_loss: 1.020 train_acc: 0.708 train_f1: 0.695 	

Validation 736 valid_acc: 0.699 best_acc: 0.699 	

Epoch 737 train_loss: 1.022 train_acc: 0.704 train_f1: 0.697 	

Validation 737 valid_acc: 0.695 best_acc: 0.699 	

Epoch 738 train_loss: 1.012 train_acc: 0.706 train_f1: 0.697 	

Validation 738 valid_acc: 0.690 best_acc: 0.699 	

Epoch 739 train_loss: 1.030 train_acc: 0.703 train_f1: 0.694 	

Validation 739 


Validation 802 valid_acc: 0.708 best_acc: 0.720 	

Epoch 803 train_loss: 0.967 train_acc: 0.723 train_f1: 0.702 	

Validation 803 valid_acc: 0.715 best_acc: 0.720 	

Epoch 804 train_loss: 0.967 train_acc: 0.726 train_f1: 0.702 	

Validation 804 valid_acc: 0.717 best_acc: 0.720 	

Epoch 805 train_loss: 0.979 train_acc: 0.721 train_f1: 0.696 	

Validation 805 valid_acc: 0.719 best_acc: 0.720 	

Epoch 806 train_loss: 0.969 train_acc: 0.725 train_f1: 0.698 	

Validation 806 valid_acc: 0.712 best_acc: 0.720 	

Epoch 807 train_loss: 0.955 train_acc: 0.728 train_f1: 0.705 	

Validation 807 valid_acc: 0.712 best_acc: 0.720 	

Epoch 808 train_loss: 0.976 train_acc: 0.722 train_f1: 0.697 	

Validation 808 valid_acc: 0.713 best_acc: 0.720 	

Epoch 809 train_loss: 0.966 train_acc: 0.723 train_f1: 0.705 	

Validation 809 valid_acc: 0.715 best_acc: 0.720 	

Epoch 810 train_loss: 0.966 train_acc: 0.726 train_f1: 0.705 	

Validation 810 valid_acc: 0.719 best_acc: 0.720 	

Epoch 811 train_loss: 0.956 


Epoch 874 train_loss: 0.932 train_acc: 0.737 train_f1: 0.705 	

Validation 874 valid_acc: 0.726 best_acc: 0.739 	

Epoch 875 train_loss: 0.939 train_acc: 0.736 train_f1: 0.703 	

Validation 875 valid_acc: 0.737 best_acc: 0.739 	

Epoch 876 train_loss: 0.933 train_acc: 0.736 train_f1: 0.700 	

Validation 876 valid_acc: 0.731 best_acc: 0.739 	

Epoch 877 train_loss: 0.928 train_acc: 0.737 train_f1: 0.708 	

Validation 877 valid_acc: 0.740 best_acc: 0.740 	

Epoch 878 train_loss: 0.923 train_acc: 0.741 train_f1: 0.709 	

Validation 878 valid_acc: 0.737 best_acc: 0.740 	

Epoch 879 train_loss: 0.926 train_acc: 0.729 train_f1: 0.699 	

Validation 879 valid_acc: 0.736 best_acc: 0.740 	

Epoch 880 train_loss: 0.939 train_acc: 0.735 train_f1: 0.704 	

Validation 880 valid_acc: 0.733 best_acc: 0.740 	

Epoch 881 train_loss: 0.931 train_acc: 0.727 train_f1: 0.698 	

Validation 881 valid_acc: 0.746 best_acc: 0.746 	

Epoch 882 train_loss: 0.921 train_acc: 0.737 train_f1: 0.703 	

Validation 882 


Validation 945 valid_acc: 0.739 best_acc: 0.757 	

Epoch 946 train_loss: 0.902 train_acc: 0.740 train_f1: 0.704 	

Validation 946 valid_acc: 0.748 best_acc: 0.757 	

Epoch 947 train_loss: 0.903 train_acc: 0.740 train_f1: 0.704 	

Validation 947 valid_acc: 0.744 best_acc: 0.757 	

Epoch 948 train_loss: 0.903 train_acc: 0.740 train_f1: 0.706 	

Validation 948 valid_acc: 0.747 best_acc: 0.757 	

Epoch 949 train_loss: 0.898 train_acc: 0.749 train_f1: 0.710 	

Validation 949 valid_acc: 0.757 best_acc: 0.757 	

Epoch 950 train_loss: 0.886 train_acc: 0.749 train_f1: 0.712 	

Validation 950 valid_acc: 0.748 best_acc: 0.757 	

Epoch 951 train_loss: 0.909 train_acc: 0.742 train_f1: 0.710 	

Validation 951 valid_acc: 0.755 best_acc: 0.757 	

Epoch 952 train_loss: 0.892 train_acc: 0.743 train_f1: 0.702 	

Validation 952 valid_acc: 0.756 best_acc: 0.757 	

Epoch 953 train_loss: 0.898 train_acc: 0.742 train_f1: 0.698 	

Validation 953 valid_acc: 0.760 best_acc: 0.760 	

Epoch 954 train_loss: 0.909 

In [3]:
print('done')
# results_directory = os.path.join(rootdir, 'results_1st'+date+'_0')
results_directory = os.path.join(rootdir, 'results_202007011stModel')

done


In [4]:
results_directory

'../results_202007011stModel'

In [5]:
# test_model = edit_ABN_model((None,12), len(unique_classes), minimum_len, n=1)

# results_directory = os.path.join(rootdir, 'results_20200622_ABN_multiclass_V4_primitiveABN')
# results_directory = os.path.join(rootdir, 'results_20200622_ABN_multiclass_V4_ABN')
# results_directory = os.path.join(rootdir, 'results_20200622_ABN_multiclass_V4_ABN_edit')
# results_directory = os.path.join(rootdir, 'results_'+date+'_ABN_primitive_n=7')
# latest_test = tf.train.latest_checkpoint(results_directory)
# test_model.load_weights(latest_test)

# test_model.summary()

In [6]:
results_directory

'../results_202007011stModel'

In [7]:
latest_test = tf.train.latest_checkpoint(results_directory)
latest_test

In [8]:
test_model =  Model(inputs=main_input, outputs=main_output)
# test_model.load_weights(latest_test)
test_model.load_weights(os.path.join(results_directory, 'ecg_mel_E700L0.16'))

test_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 72000, 12)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 72000, 12)         444       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 72000, 12)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 72000, 12)         444       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 72000, 12)         0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 36000, 12)         3468      
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 36000, 12)         0         
__________

In [9]:
final_acc, f1_classes, f1_micro,m_acc,m_f1,m_sum = test_zero(data_val, mel_directory, input_directory, class2index, minimum_len, model, x_mean_final, x_std_final)
# final_acc, f1_classes, f1_micro,m_acc,m_f1,m_sum = test(data_val, mel_directory, input_directory, class2index, minimum_len, model, x_mean_final, x_std_final)

In [10]:
final_acc

0.6666666666666666

In [11]:
f1_classes.round(3)

array([0.875, 0.733, 0.75 , 0.667, 0.   , 0.844, 0.912, 0.667, 0.444])

In [12]:
f1_micro

0.7517985611510792

In [13]:
m_acc

0.3157894736842105

In [14]:
m_f1.round(3)

array([1.   , 0.   , 0.   , 0.   , 0.   , 0.889, 0.909, 0.   , 1.   ])

In [15]:
m_sum

0.7692307692307692